In [1]:
# 定义集成预测器类 (必须在加载模型前定义，以支持pickle序列化)
# ================================================================

class EnsemblePredictor:
    """
    集成预测器类 - 用于加载和使用训练好的多个逻辑回归模型
    """
    def __init__(self, models, feature_names, threshold_strategies):
        self.models = models  # 包含model和scaler的列表
        self.feature_names = feature_names
        self.threshold_strategies = threshold_strategies
        self.n_models = len(models)
        
    def predict_proba(self, X):
        """预测概率（集成平均）"""
        if isinstance(X, pd.DataFrame):
            X = X[self.feature_names].values
        
        all_probas = []
        for model_info in self.models:
            X_scaled = model_info['scaler'].transform(X)
            proba = model_info['model'].predict_proba(X_scaled)[:, 1]
            all_probas.append(proba)
        
        return np.mean(all_probas, axis=0)
    
    def predict(self, X, threshold=0.5):
        """预测类别"""
        probas = self.predict_proba(X)
        return (probas >= threshold).astype(int)
    
    def predict_with_strategy(self, X, strategy_name):
        """使用特定策略预测"""
        probas = self.predict_proba(X)
        
        strategy_thresholds = {
            '最高召回率策略': self.threshold_strategies[0][1]['threshold'],
            '最高F1策略': self.threshold_strategies[1][1]['threshold'],
            '最高精确率策略': self.threshold_strategies[2][1]['threshold'],
            '业务平衡策略': self.threshold_strategies[3][1]['threshold']
        }
        
        threshold = strategy_thresholds.get(strategy_name, 0.5)
        return (probas >= threshold).astype(int), probas
    
    def get_feature_names(self):
        """获取特征名称"""
        return self.feature_names
    
    def get_model_info(self):
        """获取模型信息"""
        return {
            'n_models': self.n_models,
            'feature_names': self.feature_names,
            'available_strategies': ['最高召回率策略', '最高F1策略', '最高精确率策略', '业务平衡策略']
        }

print("EnsemblePredictor 类定义完成，支持pickle序列化")


EnsemblePredictor 类定义完成，支持pickle序列化


In [2]:
# 1. 导入必要的库
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
import os
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder
warnings.filterwarnings('ignore')
print("库导入完成")


库导入完成


In [3]:
# 2. 加载训练好的模型
# 注意：修改为您实际的模型路径

try:
    # 请修改为您实际的模型文件路径
    model_path = '一定要替换为你的模型路径——文件名为ensemble_predictor.pkl'
    if os.path.exists(model_path):
        # 使用joblib加载模型（推荐方式）
        trained_model = joblib.load(model_path)
        print(f"模型加载成功: {model_path}")
        print(f"模型信息: {trained_model.get_model_info()}")
    else:
        print(f"模型文件不存在: {model_path}")
        raise FileNotFoundError(f"模型文件不存在: {model_path}")
except Exception as e:
    print(f"模型加载失败: {e}")
    print(f"可能原因：")
    print(f" 1. 模型文件路径不正确")
    print(f" 2. 需要先运行EnsemblePredictor类定义")
    print(f" 3. 模型文件版本不兼容")
    raise


模型文件不存在: 一定要替换为你的模型路径——文件名为ensemble_predictor.pkl
模型加载失败: 模型文件不存在: 一定要替换为你的模型路径——文件名为ensemble_predictor.pkl
可能原因：
 1. 模型文件路径不正确
 2. 需要先运行EnsemblePredictor类定义
 3. 模型文件版本不兼容


FileNotFoundError: 模型文件不存在: 一定要替换为你的模型路径——文件名为ensemble_predictor.pkl

In [4]:
df = pd.read_parquet('外呼建模-0708.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578982 entries, 0 to 578981
Data columns (total 41 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   加密手机号码          578982 non-null  object        
 1   市场确认是否意向        578982 non-null  int64         
 2   是否转化成交          578982 non-null  int64         
 3   资源渠道            578982 non-null  object        
 4   是否接触过卓越         578982 non-null  int64         
 5   是否领过资料          578982 non-null  int64         
 6   首次成交日期_TMS      139 non-null     datetime64[ns]
 7   首次成交金额_TMS      2598 non-null    float64       
 8   首次购买科目_TMS      134 non-null     object        
 9   首次购买科数_TMS      134 non-null     float64       
 10  首次成交日期_EVIP     10 non-null      datetime64[ns]
 11  首次成交金额_EVIP     446 non-null     float64       
 12  首次购买科目_EVIP     9 non-null       object        
 13  首次购买科数_EVIP     446 non-null     float64       
 14  拨打次数            578982 non-null  int

In [6]:
# 第一部分：数据预处理与特征工程

def create_features(df):
    """创建用于机器学习的特征（使用最长通话数据）"""
    
    # 创建副本避免修改原数据
    data = df.copy()
    
    # 1. 基础特征编码
    le = LabelEncoder()
    
    # 编码分类变量
    categorical_features = ['资源渠道', 
                           'long_呼叫时段', 'long_是否工作日', 'long_周几',
                           'long_客户意向_AI', 'long_客户意向_人工']
    
    #遍历指定的类别特征列
    #用 LabelEncoder 把字符串类别转成整数编码
    for col in categorical_features:
        if col in data.columns:
            data[f'{col}_encoded'] = le.fit_transform(data[col].astype(str))
    
    # 2. 衍生特征工程
    # 通话效率特征
    # 分母添加1e-10是为了防止除0
    data['打通率'] = data['打通次数'] / (data['拨打次数'] + 1e-10)
    data['完播率'] = data['利益点完播次数'] / (data['打通次数'] + 1e-10)
    data['长通话率'] = data['通话60s以上次数'] / (data['打通次数'] + 1e-10)
    data['中等通话率'] = (data['通话15-30s次数'] + data['通话30-60s次数']) / (data['打通次数'] + 1e-10)
    
    # 通话质量特征
    data['通话质量分'] = (data['通话30-60s次数'] * 2 + data['通话60s以上次数'] * 3) / (data['打通次数'] + 1e-10)
    data['总通话时长估计'] = (
                         data['通话15-30s次数'] * 22.5 + 
                         data['通话30-60s次数'] * 45 + 
                         data['通话60s以上次数'] * 90)#使用平均值进行估计 
    
    # 客户参与度特征
    data['参与度总分'] = data['打通次数'] + data['利益点完播次数'] * 2 + data['通话60s以上次数'] * 3
    data['平均通话质量'] = data['总通话时长估计'] / (data['打通次数'] + 1e-10)
    
    # 是否为白名单
    data['是否为白名单'] = (data['资源渠道'] == '外部白名单').astype(int)
    
    
    # 基于转化用户特征创建二值化特征
    # 高于下25分位点记为高，是一种更加宽松的定义
    data['高打通率'] = (data['打通率'] > data['打通率'].quantile(0.25)).astype(int)
    data['高完播率'] = (data['完播率'] > data['完播率'].quantile(0.25)).astype(int)
    data['高长通话率'] = (data['长通话率'] > data['长通话率'].quantile(0.25)).astype(int)
    data['高参与度'] = (data['参与度总分'] > data['参与度总分'].quantile(0.25)).astype(int)
    
    # 组合特征
    data['转化模式得分'] = (data['高打通率'] + data['高完播率'] + 
                        data['高长通话率'] + data['高参与度'] + 
                        data['市场确认是否意向'])#是否使用市场确认是否意向，我个人觉得可以使用。
    
    # 4. 选择机器学习特征
    feature_columns = [
        # 基础特征
        '拨打次数', '打通次数', '利益点完播次数', '活动通知次数',
         '通话15-30s次数', '通话30-60s次数', '通话60s以上次数','long_通话时长',
        
        # 编码后的分类特征
        '资源渠道_encoded',
        'long_呼叫时段_encoded', 'long_是否工作日_encoded', 'long_周几_encoded',
        'long_客户意向_AI_encoded', 'long_客户意向_人工_encoded',
        
        # 衍生特征
        '打通率', '完播率', '长通话率', '中等通话率',
        '通话质量分', '总通话时长估计', '参与度总分', '平均通话质量',
        '是否为白名单'
        # 
        '高打通率', '高完播率', '高长通话率', '高参与度', '转化模式得分'
    ]
    
    # 过滤存在的特征
    available_features = [col for col in feature_columns if col in data.columns]
    
    X = data[available_features]

    y = data['是否转化成交']
    #通过这里的替换可以建模意向，注意上面的转化模式得分就要把是否意向去掉
    #y = data['市场确认是否意向']
    
    # 处理缺失值
    X = X.fillna(0)

    print(f"总特征数量: {len(available_features)}")
    print(f"新增转化区分特征: 5个")
    
    return X, y, available_features, data

# 执行特征工程
X, y, feature_names, processed_data = create_features(df)

print(f"特征数量: {X.shape[1]}")
print(f"样本数量: {X.shape[0]}")
print(f"正负样本比例: {(y==0).sum()}:{(y==1).sum()} = {(y==0).sum()/(y==1).sum():.1f}:1")


总特征数量: 26
新增转化区分特征: 5个
特征数量: 26
样本数量: 578982
正负样本比例: 578833:149 = 3884.8:1


In [7]:
# 6. 模型预测
# 预测转化概率
X_new = X
try:
    # 使用EnsemblePredictor的预测方法
    #X_new_scaled = trained_model['scaler'].transform(X_new)
    prediction_probabilities = trained_model.predict_proba(X_new)
    #prediction_probabilities = trained_model['model'].predict_proba(X_new_scaled)[:, 1]
    print(f"使用集成模型预测完成")
    print(f"模型包含 {trained_model.n_models} 个子模型")
    print(f"使用特征: {trained_model.get_feature_names()}")
    print(f"预测完成，预测了{len(prediction_probabilities)}个用户的转化概率")
    print(f"概率范围: {prediction_probabilities.min():.3f} - {prediction_probabilities.max():.3f}")
    print(f"平均概率: {prediction_probabilities.mean():.3f}")
    
except Exception as e:
    print(f"预测失败: {e}")
    raise


使用集成模型预测完成
模型包含 100 个子模型
使用特征: ['拨打次数', '打通次数', '利益点完播次数', '活动通知次数', '通话15-30s次数', '通话30-60s次数', '通话60s以上次数', 'long_通话时长', '资源渠道_encoded', 'long_呼叫时段_encoded', 'long_是否工作日_encoded', 'long_周几_encoded', 'long_客户意向_AI_encoded', 'long_客户意向_人工_encoded', '打通率', '完播率', '长通话率', '中等通话率', '通话质量分', '总通话时长估计', '参与度总分', '平均通话质量', '高完播率', '高长通话率', '高参与度', '转化模式得分']
预测完成，预测了578982个用户的转化概率
概率范围: 0.000 - 1.000
平均概率: 0.030


In [ ]:
processed_new_data=df
# 7. 准备Excel输出数据
# 创建详细的用户概率表
user_probability_table = pd.DataFrame({
    '用户ID': processed_new_data['加密手机号码'],
    '转化概率': prediction_probabilities,
    '概率百分比': (prediction_probabilities * 100).round(1),
})
final_output_df = user_probability_table

print(f"数据整理完成，准备保存Excel文件")
# 将 final_output_df 和 df 通过“加密手机号”连接
merged_df = final_output_df.merge(
    df, 
    left_on='用户ID', 
    right_on='加密手机号码', 
    how='left'  # or 'inner' if只保留交集
)


✅ 数据整理完成，准备保存Excel文件


In [8]:
merged_df.drop(columns=["用户ID"], inplace=True)
merged_df

,转化概率,概率百分比,加密手机号码,市场确认是否意向,是否转化成交,资源渠道,是否接触过卓越,是否领过资料,首次成交日期_TMS,首次成交金额_TMS,...,latest_通话时长,latest_呼叫时段,latest_是否工作日,latest_周几,latest_通话活动名称,latest_客户意向_AI,latest_客户意向_人工,客户属性文本汇总,首次呼叫时间,首次打通时间
0,0.007412,0.7,5892fb0b804b7f50761cc1f0fedc10b6,0,0,外部白名单,0,0,NaT,NaN,...,0,15点,是,周二,5月研学活动,E(未接通),E(未接通),|广州市越秀区红火炬小学|越秀|tmk资源池|S7,2025-03-25 19:11:15,2025-03-25 19:11:15
1,0.199076,19.9,589389297cc3c0cb08d010541afa9911,0,0,亲子沉睡激活,1,1,NaT,NaN,...,0,19点,是,周四,【悦学广州分校】3-20高三政策咨询,E(未接通),E(未接通),"|机场西（飞翔公园）|白云分区|市场营运,|利益点完拨率100%|机场西（飞翔公园）|白云分...",2025-02-09 16:48:52,2025-02-09 17:22:19
2,0.007541,0.8,5893ada6d8874b40d77994d5da47c6a4,0,0,外部白名单,0,0,NaT,NaN,...,0,19点,是,周五,5月邀约期末真题集（tctm简短）,E(未接通),E(未接通),|无基础|未知|用户增长处|花都分区|S1,2025-02-06 16:21:25,2025-02-06 16:21:25
3,0.009273,0.9,5893bb8b4e6dcb5101d654f379918f7d,0,0,外部白名单,0,0,NaT,NaN,...,0,16点,是,周四,【悦学广州分校】2月6号邀约TCTM公益活动,E(未接通),E(未接通),|越秀区，东风东校区（东峻广场3楼）|广图B|外部渠道tctm|广州公园前|J2,2025-02-06 16:01:37,2025-02-06 16:01:37
4,0.010108,1.0,5893f192716215e7ec1473a8f00489b8,0,0,成长沉睡激活,1,1,NaT,NaN,...,0,19点,是,周三,【悦学广州分校】2月14号邀约成长素养体验课,E(未接通),E(未接通),|车陂|天秀分区,2025-02-16 16:25:23,2025-02-16 16:25:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578977,0.008061,0.8,589288dcdb83c9680ce46e21141a3d15,0,0,成长沉睡激活,1,1,NaT,NaN,...,0,19点,是,周四,【悦学广州分校】4-17邀约趣味拼音,E(未接通),E(未接通),|广图B|外部渠道tctm|广州公园前|J2,2025-04-17 19:44:28,2025-04-17 19:44:28
578978,0.008920,0.9,58929204ee730aee9748a94703beac44,0,0,外部白名单,0,0,NaT,NaN,...,0,19点,是,周二,5月邀约素养课程（新4年级异业）,E(未接通),E(未接通),|用户增长处|番禺区|S3,2025-05-27 19:12:24,2025-05-27 19:12:24
578979,0.008385,0.8,5892be1a74d4f4f65da1db51979c9fc4,0,0,成长市场,1,1,NaT,NaN,...,0,19点,是,周四,4-19志愿填报见面会,F(停机/空号),F(停机/空号),|成长市场|白云分区|S9,2025-04-24 19:40:36,2025-04-24 19:40:36
578980,0.001606,0.2,5892d499799f2d1182b0db8d1f19312b,0,0,tmk资源池,1,1,NaT,NaN,...,7,15点,是,周二,5月研学活动,L(短秒挂断),L(短秒挂断),|广州开发区第一小学|黄埔|萝岗香雪B|成长市场|黄埔分区|S6,2024-12-26 14:34:22,2024-12-26 14:34:22


In [ ]:
merged_df.to_excel('全量预测结果f2优化.xlsx', index=False)